In [94]:
import mesa
import numpy as np
# from mesa.time import RandomActivation
from mesa.space import MultiGrid
# 导入ABMSimulator instance for event scheduling
from mesa.experimental.devs import ABMSimulator

In [95]:
# 中央政府智能体类
class CentralGovernment(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        self.reproductive_subsidy = 1000  # 生育补贴标准
        self.retirement_age = 65  # 退休年龄
        self.transfer_ratio = 0.1  # 财政转移支付比例

    def step(self):
        pass


In [96]:

# 地方政府智能体类
class LocalGovernment(mesa.Agent):
    def __init__(self, model, region):
        super().__init__( model)
        self.region = region
        self.extra_subsidy = 500  # 额外补贴
        self.fiscal_income = 0
        self.fiscal_expenditure = 0

    def step(self):
        # 执行中央政策并附加地方性措施
        pass

In [97]:

# 居民智能体类
class Resident(mesa.Agent):
    def __init__(self, model, region):
        super().__init__( model)
        self.region = region
        self.salary = np.random.randint(2000, 8000)

    def step(self):
        # 跨区域迁移决策
        regions = self.model.regions
        current_region = self.region
        for region in regions:
            if region != current_region:
                wj = self.model.get_average_wage(region)
                wi = self.model.get_average_wage(current_region)
                k = 0.1  # 迁移敏感系数
                migration_prob = k * (wj - wi) / wi
                if np.random.random() < migration_prob:
                    self.region = region
                    self.model.move_resident(self, current_region, region)


In [98]:

# 企业智能体类
class Enterprise(mesa.Agent):
    def __init__(self, model, region):
        super().__init__(model)
        self.region = region
        self.labor_cost = self.model.get_average_wage(region)
        self.recruitment_strategy = 10  # 初始招聘策略

    def step(self):
        # 根据区域劳动力成本调整招聘策略
        current_labor_cost = self.model.get_average_wage(self.region)
        if current_labor_cost > self.labor_cost:
            self.recruitment_strategy -= 1
        elif current_labor_cost < self.labor_cost:
            self.recruitment_strategy += 1
        self.labor_cost = current_labor_cost


In [99]:
# 模型类
class NationalModel(mesa.Model):
    def __init__(self, num_regions, num_residents, num_enterprises ,simulator:ABMSimulator = None):
        # 添加一行
        super().__init__(seed=None)
        # 添加两行和对应参数
        # self.simulator = simulator
        # self.simulator.setup(self)
        self.num_regions = num_regions
        # self.schedule = RandomActivation(self)
        # 可能要改
        self.grid = MultiGrid(10, 10, torus=True)
        self.regions = list(range(num_regions))
        self.residents = []
        self.enterprises = []

        # 创建中央政府智能体
        self.central_government = CentralGovernment(model=self)
        # self.schedule.add(self.central_government)

        # 创建地方政府智能体
        for i in range(num_regions):
            local_gov = LocalGovernment( self, i)
            # self.schedule.add(local_gov)

        # 创建居民智能体
        for i in range(num_residents):
            region = np.random.choice(self.regions)
            resident = Resident( self, region)
            self.residents.append(resident)
            # self.schedule.add(resident)

        # 创建企业智能体
        for i in range(num_enterprises):
            region = np.random.choice(self.regions)
            enterprise = Enterprise( self, region)
            self.enterprises.append(enterprise)
            # self.schedule.add(enterprise)

    def get_average_wage(self, region):
        residents_in_region = [r for r in self.residents if r.region == region]
        if len(residents_in_region) == 0:
            return 0
        total_salary = sum([r.salary for r in residents_in_region])
        return total_salary / len(residents_in_region)

    def move_resident(self, resident, from_region, to_region):
        # 处理居民迁移的逻辑
        pass

    def calculate_labor_flow(self):
        # 计算劳动力流动
        for i in self.regions:
            for j in self.regions:
                if i != j:
                    wi = self.get_average_wage(i)
                    wj = self.get_average_wage(j)
                    Li = len([r for r in self.residents if r.region == i])
                    k = 0.1
                    Mi_j = k * (wj - wi) / wi * Li
                    print(f"从区域 {i} 到区域 {j} 的迁移人口: {Mi_j}")

    def calculate_fiscal_transfer(self):
        # 计算财政转移支付
        central_gov = self.central_government
        for i in self.regions:
            for j in self.regions:
                if i != j:
                    GDPi = np.random.randint(10000, 50000)
                    GDPj = np.random.randint(10000, 50000)
                    transfer = central_gov.transfer_ratio * (GDPi - GDPj)
                    print(f"从区域 {i} 到区域 {j} 的财政转移支付: {transfer}")

    def step(self):
        # self.schedule.step()
        # 添加一行
        self.agents.shuffle_do("step")
        self.calculate_labor_flow()
        self.calculate_fiscal_transfer()


In [100]:
# 创建模型实例并运行模拟
model = NationalModel(num_regions=3, num_residents=100, num_enterprises=20)
for _ in range(10):
    model.step()
    

从区域 0 到区域 1 的迁移人口: 0.18872988569122384
从区域 0 到区域 2 的迁移人口: 0.2253881738032052
从区域 1 到区域 0 的迁移人口: -0.18378821676251716
从区域 1 到区域 2 的迁移人口: 0.035698434177459645
从区域 2 到区域 0 的迁移人口: -0.23029758038644396
从区域 2 到区域 1 的迁移人口: -0.03745677916832385
从区域 0 到区域 1 的财政转移支付: -423.0
从区域 0 到区域 2 的财政转移支付: -1146.8
从区域 1 到区域 0 的财政转移支付: -1933.1000000000001
从区域 1 到区域 2 的财政转移支付: -92.30000000000001
从区域 2 到区域 0 的财政转移支付: 851.7
从区域 2 到区域 1 的财政转移支付: 2390.0
从区域 0 到区域 1 的迁移人口: 0.18872988569122384
从区域 0 到区域 2 的迁移人口: 0.2253881738032052
从区域 1 到区域 0 的迁移人口: -0.18378821676251716
从区域 1 到区域 2 的迁移人口: 0.035698434177459645
从区域 2 到区域 0 的迁移人口: -0.23029758038644396
从区域 2 到区域 1 的迁移人口: -0.03745677916832385
从区域 0 到区域 1 的财政转移支付: -842.9000000000001
从区域 0 到区域 2 的财政转移支付: 207.70000000000002
从区域 1 到区域 0 的财政转移支付: -1293.9
从区域 1 到区域 2 的财政转移支付: 975.6
从区域 2 到区域 0 的财政转移支付: -446.0
从区域 2 到区域 1 的财政转移支付: -1241.7
从区域 0 到区域 1 的迁移人口: 0.22721662392544228
从区域 0 到区域 2 的迁移人口: 0.2931537681349253
从区域 1 到区域 0 的迁移人口: -0.22535799249203367
从区域 1 到区域 2 的迁移人口: 0.06

In [101]:
# import mesa
# import numpy as np
# from mesa.space import MultiGrid

# # 中央政府智能体类
# class CentralGovernment(mesa.Agent):
#     def __init__(self,  model):
#         super().__init__( model)
#         self.reproductive_subsidy = 1000  # 生育补贴标准
#         self.retirement_age = 65  # 退休年龄
#         self.transfer_ratio = 0.1  # 财政转移支付比例

#     def step(self):
#         pass

# # 地方政府智能体类
# class LocalGovernment(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.extra_subsidy = 500  # 额外补贴
#         self.fiscal_income = 0
#         self.fiscal_expenditure = 0

#     def step(self):
#         # 执行中央政策并附加地方性措施
#         pass

# # 居民智能体类
# class Resident(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.salary = np.random.randint(2000, 8000)

#     def step(self):
#         # 跨区域迁移决策
#         regions = self.model.regions
#         current_region = self.region
#         for region in regions:
#             if region != current_region:
#                 wj = self.model.get_average_wage(region)
#                 wi = self.model.get_average_wage(current_region)
#                 k = 0.1  # 迁移敏感系数
#                 migration_prob = k * (wj - wi) / wi
#                 if np.random.random() < migration_prob:
#                     self.region = region
#                     self.model.move_resident(self, current_region, region)

# # 企业智能体类
# class Enterprise(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.labor_cost = self.model.get_average_wage(region)
#         self.recruitment_strategy = 10  # 初始招聘策略

#     def step(self):
#         # 根据区域劳动力成本调整招聘策略
#         current_labor_cost = self.model.get_average_wage(self.region)
#         if current_labor_cost > self.labor_cost:
#             self.recruitment_strategy -= 1
#         elif current_labor_cost < self.labor_cost:
#             self.recruitment_strategy += 1
#         self.labor_cost = current_labor_cost

# # 模型类
# class NationalModel(mesa.Model):
#     def __init__(self, num_regions, num_residents, num_enterprises):
#         self.num_regions = num_regions
#         self.schedule = mesa.RandomActivation(self)
#         self.grid = MultiGrid(10, 10, torus=True)
#         self.regions = list(range(num_regions))
#         self.residents = []
#         self.enterprises = []

#         # 创建中央政府智能体
#         self.central_government = CentralGovernment(0, self)
#         self.schedule.add(self.central_government)

#         # 创建地方政府智能体
#         for i in range(num_regions):
#             local_gov = LocalGovernment(i + 1, self, i)
#             self.schedule.add(local_gov)

#         # 创建居民智能体
#         for i in range(num_residents):
#             region = np.random.choice(self.regions)
#             resident = Resident(i + num_regions + 1, self, region)
#             self.residents.append(resident)
#             self.schedule.add(resident)

#         # 创建企业智能体
#         for i in range(num_enterprises):
#             region = np.random.choice(self.regions)
#             enterprise = Enterprise(i + num_regions + num_residents + 1, self, region)
#             self.enterprises.append(enterprise)
#             self.schedule.add(enterprise)

#     def get_average_wage(self, region):
#         residents_in_region = [r for r in self.residents if r.region == region]
#         if len(residents_in_region) == 0:
#             return 0
#         total_salary = sum([r.salary for r in residents_in_region])
#         return total_salary / len(residents_in_region)

#     def move_resident(self, resident, from_region, to_region):
#         # 处理居民迁移的逻辑
#         pass

#     def calculate_labor_flow(self):
#         # 计算劳动力流动
#         for i in self.regions:
#             for j in self.regions:
#                 if i != j:
#                     wi = self.get_average_wage(i)
#                     wj = self.get_average_wage(j)
#                     Li = len([r for r in self.residents if r.region == i])
#                     k = 0.1
#                     Mi_j = k * (wj - wi) / wi * Li
#                     print(f"从区域 {i} 到区域 {j} 的迁移人口: {Mi_j}")

#     def calculate_fiscal_transfer(self):
#         # 计算财政转移支付
#         central_gov = self.central_government
#         for i in self.regions:
#             for j in self.regions:
#                 if i != j:
#                     GDPi = np.random.randint(10000, 50000)
#                     GDPj = np.random.randint(10000, 50000)
#                     transfer = central_gov.transfer_ratio * (GDPi - GDPj)
#                     print(f"从区域 {i} 到区域 {j} 的财政转移支付: {transfer}")

#     def step(self):
#         self.schedule.step()
#         self.calculate_labor_flow()
#         self.calculate_fiscal_transfer()

# # 创建模型实例并运行模拟
# model = NationalModel(num_regions=3, num_residents=100, num_enterprises=20)
# for _ in range(10):
#     model.step()
    

In [102]:
# import mesa
# import numpy as np
# from mesa.time import RandomActivation
# from mesa.space import MultiGrid

# # 中央政府智能体类
# class CentralGovernment(mesa.Agent):
#     def __init__(self, unique_id, model):
#         super().__init__(unique_id, model)
#         self.reproductive_subsidy = 1000  # 生育补贴标准
#         self.retirement_age = 65  # 退休年龄
#         self.transfer_ratio = 0.1  # 财政转移支付比例

#     def step(self):
#         pass

# # 地方政府智能体类
# class LocalGovernment(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.extra_subsidy = 500  # 额外补贴
#         self.fiscal_income = 0
#         self.fiscal_expenditure = 0

#     def step(self):
#         # 执行中央政策并附加地方性措施
#         pass

# # 居民智能体类
# class Resident(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.salary = np.random.randint(2000, 8000)

#     def step(self):
#         # 跨区域迁移决策
#         regions = self.model.regions
#         current_region = self.region
#         for region in regions:
#             if region != current_region:
#                 wj = self.model.get_average_wage(region)
#                 wi = self.model.get_average_wage(current_region)
#                 k = 0.1  # 迁移敏感系数
#                 migration_prob = k * (wj - wi) / wi
#                 if np.random.random() < migration_prob:
#                     self.region = region
#                     self.model.move_resident(self, current_region, region)

# # 企业智能体类
# class Enterprise(mesa.Agent):
#     def __init__(self, unique_id, model, region):
#         super().__init__(unique_id, model)
#         self.region = region
#         self.labor_cost = self.model.get_average_wage(region)
#         self.recruitment_strategy = 10  # 初始招聘策略

#     def step(self):
#         # 根据区域劳动力成本调整招聘策略
#         current_labor_cost = self.model.get_average_wage(self.region)
#         if current_labor_cost > self.labor_cost:
#             self.recruitment_strategy -= 1
#         elif current_labor_cost < self.labor_cost:
#             self.recruitment_strategy += 1
#         self.labor_cost = current_labor_cost

# # 模型类
# class NationalModel(mesa.Model):
#     def __init__(self, num_regions, num_residents, num_enterprises):
#         self.num_regions = num_regions
#         self.schedule = RandomActivation(self)
#         self.grid = MultiGrid(10, 10, torus=True)
#         self.regions = list(range(num_regions))
#         self.residents = []
#         self.enterprises = []

#         # 创建中央政府智能体
#         self.central_government = CentralGovernment(0, self)
#         self.schedule.add(self.central_government)

#         # 创建地方政府智能体
#         for i in range(num_regions):
#             local_gov = LocalGovernment(i + 1, self, i)
#             self.schedule.add(local_gov)

#         # 创建居民智能体
#         for i in range(num_residents):
#             region = np.random.choice(self.regions)
#             resident = Resident(i + num_regions + 1, self, region)
#             self.residents.append(resident)
#             self.schedule.add(resident)

#         # 创建企业智能体
#         for i in range(num_enterprises):
#             region = np.random.choice(self.regions)
#             enterprise = Enterprise(i + num_regions + num_residents + 1, self, region)
#             self.enterprises.append(enterprise)
#             self.schedule.add(enterprise)

#     def get_average_wage(self, region):
#         residents_in_region = [r for r in self.residents if r.region == region]
#         if len(residents_in_region) == 0:
#             return 0
#         total_salary = sum([r.salary for r in residents_in_region])
#         return total_salary / len(residents_in_region)

#     def move_resident(self, resident, from_region, to_region):
#         # 处理居民迁移的逻辑
#         pass

#     def calculate_labor_flow(self):
#         # 计算劳动力流动
#         for i in self.regions:
#             for j in self.regions:
#                 if i != j:
#                     wi = self.get_average_wage(i)
#                     wj = self.get_average_wage(j)
#                     Li = len([r for r in self.residents if r.region == i])
#                     k = 0.1
#                     Mi_j = k * (wj - wi) / wi * Li
#                     print(f"从区域 {i} 到区域 {j} 的迁移人口: {Mi_j}")

#     def calculate_fiscal_transfer(self):
#         # 计算财政转移支付
#         central_gov = self.central_government
#         for i in self.regions:
#             for j in self.regions:
#                 if i != j:
#                     GDPi = np.random.randint(10000, 50000)
#                     GDPj = np.random.randint(10000, 50000)
#                     transfer = central_gov.transfer_ratio * (GDPi - GDPj)
#                     print(f"从区域 {i} 到区域 {j} 的财政转移支付: {transfer}")

#     def step(self):
#         self.schedule.step()
#         self.calculate_labor_flow()
#         self.calculate_fiscal_transfer()

# # 创建模型实例并运行模拟
# model = NationalModel(num_regions=3, num_residents=100, num_enterprises=20)
# for _ in range(10):
#     model.step()
    